In [1]:
# 1. (推荐) 设置 iPython 自动重载
#    这能让您在修改 .py 文件后，Notebook 能自动加载最新版本
#    无需重启内核
%load_ext autoreload
%autoreload 2

# 2. (关键) 更改当前工作目录 (Change Directory)
#    '..' 代表上一级目录
#    执行后，您的 Notebook 将会从"项目根目录"运行
%cd ..

import arcpy

# 导入您的 SoilSampler 类
# (现在可以无缝工作了)
from script.Function import SoilSampler

e:\Soil\SamplingPoint


In [2]:
# ==========================================================
# 1. 设置您的参数
# ==========================================================

# 您的工作空间 GDB
WORKSPACE_GDB = r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb"

# 您的批量配置
MY_BATCH_CONFIG = [
    {
        "id": "Region_A",
        "points": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_A_peaks_point",
        "farmland": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_A"
    },
    {
        "id": "Region_B",
        "points": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_B_peaks_point",
        "farmland": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_B"
    },
    {
        "id": "Region_C",
        "points": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_C_peaks_point",
        "farmland": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_C"
    },
]

# 您研究区域的正确 UTM 坐标系名称或 WKID
# (这是最关键的参数，必须正确设置)
# 例如：
#   - "WGS 1984 UTM Zone 50N" (适用于北京附近)
#   - "WGS 1984 UTM Zone 49N" (适用于广东附近)
#   - 32650 (这是 UTM Zone 50N 的 WKID)

# (关键) 您研究区域的正确 UTM 坐标系
UTM_COORDINATE_SYSTEM = "WGS 1984 UTM Zone 50N" # !! <== 请根据您的位置修改此项 !!

# (改进 4) 您的 .lyrx 样式模板路径 (请先在 Pro 中手动创建好)
# 这是一个按 "class" 字段 (cold, normal, hot) 进行唯一值渲染的图层文件
SYMBOLOGY_TEMPLATE_PATH = r"E:\Documents\ArcGIS\Projects\SamplingPoint\sample_points_classified_style.lyrx" # <== 修改此路径


# ==========================================================
# 2. 检查 ArcPy 许可
# ==========================================================
if arcpy.CheckExtension("Spatial") != "Available":
    raise Exception("未能获取 Spatial Analyst 扩展许可。")
else:
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst 许可已检出。")

# ==========================================================
# 3. 实例化并运行工具
# ==========================================================
try:
    print("开始初始化土壤采样器 (v2.0)...")
    
    # --- 实例化 SoilSampler 类 (v2.0) ---
    # 您现在可以传入所有新参数来自定义您的工作流
    
    sampler = SoilSampler(
        workspace=WORKSPACE_GDB,
        batch_config=MY_BATCH_CONFIG,
        output_utm_sr=UTM_COORDINATE_SYSTEM,
        
        # --- (改进 1) 自定义模型参数 ---
        kriging_model_str="SPHERICAL", # 例如，改用指数模型，可选参数有： 'SPHERICAL', 'EXPONENTIAL', 'GAUSSIAN', 'LINEAR', 'CIRCULAR'
        kriging_range=757.38, # 地统计想到给出的范围参数 (以米为单位)
        line_point_distance="10 Meters", # 例如，使用更稀疏的航线点
        
        # --- (改进 2) 自定义分类方法 ---
        # 选项 1: (默认) 'quantile'
        classification_method="quantile",
        
        # # 选项 2: 'std_dev' (标准差)
        # classification_method="std_dev",
        # classification_std_dev_factor=0.75, # 使用 0.75 倍标准差
        
        # 选项 3: 'manual' (手动)
        # classification_method="manual",
        # classification_thresholds={"p33": 1000, "p66": 3000},

        # --- (改进 3) 自定义采样策略 ---
        min_distance=100, # 首选 100 米
        sample_config={
             "coldPoint": 5,
             "normalPoint": 3,
             "hotPoint": 5,
         },
        # (使用自动回退 [150, 75, 0])
        # sampling_fallback_distances=[150, 100, 50, 0], # 或者手动指定回退
        
        # --- (改进 4) 符号系统 ---
        symbology_template_lyrx=SYMBOLOGY_TEMPLATE_PATH
    )
    
    # 运行批量处理
    print("初始化成功。开始运行批量处理...")
    sampler.run_batch()

except Exception as e:
    print(f"\n!! 脚本执行时遇到致命错误: {e}")

finally:
    # 归还许可
    arcpy.CheckInExtension("Spatial")
    print("脚本执行完毕，已归还 Spatial Analyst 许可。")

Spatial Analyst 许可已检出。
开始初始化土壤采样器 (v2.0)...
工作空间设置为: E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb
使用用户指定的坐标系: WGS_1984_UTM_Zone_50N
初始化成功。开始运行批量处理...
开始批量处理 3 个区域...
采样配置: {'coldPoint': 5, 'normalPoint': 3, 'hotPoint': 5}
分类方法: quantile
采样距离策略: [100, 50.0, 0]

正在处理区域: Region_A
  输入点: E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_A_peaks_point
  农田范围: E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_A
  步骤 1: 克里金插值 (模型: SPHERICAL, 变程: 757.38)...
    输出: kriging_total_Region_A
  步骤 2: 点转线...
  步骤 3: 沿线生成点 (距离: 10 Meters)...
  步骤 4: 值提取至点...
  步骤 5: 分类点 (方法: quantile)...
    阈值 (分位数): < 122729.33 (cold), < 123372.45 (normal)
  步骤 5b: 投影航线点至 WGS_1984_UTM_Zone_50N...
    输出: route_points_UTM_Region_A
  步骤 6: 采样点 (首选距离 100m)...
    正在采样 5 个 'coldPoint' 点...
    ...正在尝试 100m 最小距离...
    成功：在 100m 距离下抽出 5 个点。
    正在采样 3 个 'normalPoint' 点...
    ...正在尝试 100m 最小距离...
    成功：在 100m 距离下抽出 3 个点。
    正在采样 5 个 'hotPoint' 点...
    .